# Ollama + OpenAI + Python

## 1. Nurodykite modelio pavadinimą

Jei pasirinktumėte kitą modelį nei "phi3:mini", pakeiskite reikšmę žemiau esančiame langelyje.
Šis kintamasis bus naudojamas kode visoje užrašų knygutėje.


In [ ]:
MODEL_NAME = "phi3:mini"

## 2. Nustatykite Open AI klientą

Įprastai Open AI klientas naudojamas su OpenAI.com arba Azure OpenAI, kad bendrautų su dideliais kalbos modeliais.
Tačiau jis taip pat gali būti naudojamas su Ollama, nes Ollama pateikia OpenAI suderinamą galinį tašką adresu "http://localhost:11434/v1".


In [ ]:
%pip install openai

In [ ]:
import openai

client = openai.OpenAI(
    base_url="http://localhost:11434/v1",
    api_key="nokeyneeded",
)

## 3. Generuoti pokalbio atsakymą

Dabar galime naudoti OpenAI SDK, kad sugeneruotume atsakymą pokalbiui. Šis užklausimas turėtų sugeneruoti haiku apie kates:


In [ ]:
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "Write a haiku about a hungry cat"},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 4. Užklausų inžinerija

Pirmasis pranešimas, siunčiamas kalbų modeliui, vadinamas „sistemos pranešimu“ arba „sistemos užklausa“ ir jis nustato bendrąsias modelio instrukcijas.
Galite pateikti savo pasirinktą sistemos užklausą, kad nukreiptumėte kalbų modelį generuoti išvestį kitaip.
Modify the `SYSTEM_MESSAGE` below to answer like your favorite famous movie/TV character, or get inspiration for other system prompts from [Awesome ChatGPT Prompts](https://github.com/f/awesome-chatgpt-prompts?tab=readme-ov-file#prompts).

Once you've customized the system message, provide the first user question in the `USER_MESSAGE`.


In [ ]:
SYSTEM_MESSAGE = """
I want you to act like Elmo from Sesame Street.
I want you to respond and answer like Elmo using the tone, manner and vocabulary that Elmo would use.
Do not write any explanations. Only answer like Elmo.
You must know all of the knowledge of Elmo, and nothing more.
"""

USER_MESSAGE = """
Hi Elmo, how are you doing today?
"""

response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE},
    ],
)

print("Response:")
print(response.choices[0].message.content)


## 5. Few-shot pavyzdžiai

Kitas būdas nukreipti kalbos modelį yra pateikti „few-shot“ pavyzdžių seką — eilę pavyzdinių klausimų/atsakymų, kurie demonstruoja, kaip jis turėtų atsakyti.

Žemiau pateiktas pavyzdys bando priversti kalbos modelį elgtis kaip dėstytojo asistentas (TA), pateikdamas kelis klausimų ir atsakymų pavyzdžius, kuriuos TA galėtų pateikti, o tada užduodamas modeliui klausimą, kurį galėtų užduoti studentas.

Išbandykite tai pirmiausia, o tada pakeiskite `SYSTEM_MESSAGE`, `EXAMPLES`, ir `USER_MESSAGE` naujam scenarijui.


In [ ]:
SYSTEM_MESSAGE = """
You are a helpful assistant that helps students with their homework.
Instead of providing the full answer, you respond with a hint or a clue.
"""

EXAMPLES = [
    (
        "What is the capital of France?",
        "Can you remember the name of the city that is known for the Eiffel Tower?"
    ),
    (
        "What is the square root of 144?",
        "What number multiplied by itself equals 144?"
    ),
    (   "What is the atomic number of oxygen?",
        "How many protons does an oxygen atom have?"
    ),
]

USER_MESSAGE = "What is the largest planet in our solar system?"


response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": EXAMPLES[0][0]},
        {"role": "assistant", "content": EXAMPLES[0][1]},
        {"role": "user", "content": EXAMPLES[1][0]},
        {"role": "assistant", "content": EXAMPLES[1][1]},
        {"role": "user", "content": EXAMPLES[2][0]},
        {"role": "assistant", "content": EXAMPLES[2][1]},
        {"role": "user", "content": USER_MESSAGE},
    ],
)


print("Response:")
print(response.choices[0].message.content)

## 6. Paieškos papildyta generacija

RAG (Retrieval Augmented Generation) yra technika, leidžianti kalbos modeliui tiksliai atsakyti į klausimus tam tikroje srityje, pirmiausia išgaunant susijusią informaciją iš žinių šaltinio, o tada generuojant atsakymą remiantis ta informacija.

Mes pateikėme vietinį CSV failą su duomenimis apie hibridinius automobilius. Žemiau pateiktas kodas nuskaito CSV failą, ieško atitikmenų pagal vartotojo klausimą ir tuomet sugeneruoja atsakymą remdamasis rastais duomenimis. Atkreipkite dėmesį, kad tai užtruks ilgiau nei bet kuris ankstesnis pavyzdys, nes modeliui siunčiama daugiau duomenų. Jei pastebite, kad atsakymas vis tiek nėra pagrįstas duomenimis, galite pabandyti pakeisti sistemos inžineriją arba išbandyti kitus modelius. Apskritai RAG yra efektyvesnis su didesniais modeliais arba su tikslingai pritaikytomis SLM versijomis.


In [ ]:
import csv

SYSTEM_MESSAGE = """
You are a helpful assistant that answers questions about cars based off a hybrid car data set.
You must use the data set to answer the questions, you should not provide any information that is not in the provided sources.
"""

USER_MESSAGE = "how fast is a prius?"

# Open the CSV and store in a list
with open("hybrid.csv", "r") as file:
    reader = csv.reader(file)
    rows = list(reader)

# Normalize the user question to replace punctuation and make lowercase
normalized_message = USER_MESSAGE.lower().replace("?", "").replace("(", " ").replace(")", " ")

# Search the CSV for user question using very naive search
words = normalized_message.split()
matches = []
for row in rows[1:]:
    # if the word matches any word in row, add the row to the matches
    if any(word in row[0].lower().split() for word in words) or any(word in row[5].lower().split() for word in words):
        matches.append(row)

# Format as a markdown table, since language models understand markdown
matches_table = " | ".join(rows[0]) + "\n" + " | ".join(" --- " for _ in range(len(rows[0]))) + "\n"
matches_table += "\n".join(" | ".join(row) for row in matches)
print(f"Found {len(matches)} matches:")
print(matches_table)

# Now we can use the matches to generate a response
response = client.chat.completions.create(
    model=MODEL_NAME,
    temperature=0.7,
    n=1,
    messages=[
        {"role": "system", "content": SYSTEM_MESSAGE},
        {"role": "user", "content": USER_MESSAGE + "\nSources: " + matches_table},
    ],
)

print("Response:")
print(response.choices[0].message.content)

---

<!-- CO-OP TRANSLATOR DISCLAIMER START -->
Atsakomybės apribojimas:
Šis dokumentas buvo išverstas naudojant dirbtinio intelekto vertimo paslaugą Co-op Translator (https://github.com/Azure/co-op-translator). Nors siekiame užtikrinti tikslumą, prašome atkreipti dėmesį, kad automatizuoti vertimai gali turėti klaidų ar netikslumų. Originalus dokumentas jo gimtąja kalba turėtų būti laikomas autoritetingu šaltiniu. Kritinei informacijai rekomenduojamas profesionalus, žmogaus atliekamas vertimas. Mes neprisiimame atsakomybės už bet kokius nesusipratimus ar neteisingas interpretacijas, kilusias dėl šio vertimo naudojimo.
<!-- CO-OP TRANSLATOR DISCLAIMER END -->
